# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: use Lora config
* Model: GPT-2
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [6]:
# Load dataset, and split into train and test

from datasets import load_dataset

splits = ['train', 'test']
ds = {split: ds for split, ds in zip(splits, load_dataset('imdb', split=splits))}

for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))
    
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [7]:
# Import tokenizer to convert strings into tokens
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    print('initialize pad token...')
    tokenizer.pad_token = tokenizer.eos_token
    
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess, batched=True)

# print first example of tokenized training set
print(tokenized_ds['train'][0]["input_ids"])

initialize pad token...


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[1858, 318, 645, 8695, 379, 477, 1022, 6401, 959, 290, 4415, 5329, 475, 262, 1109, 326, 1111, 389, 1644, 2168, 546, 6590, 6741, 13, 4415, 5329, 3073, 42807, 11, 6401, 959, 3073, 6833, 13, 4415, 5329, 21528, 389, 2407, 2829, 13, 6401, 959, 338, 7110, 389, 1290, 517, 8253, 986, 6401, 959, 3073, 517, 588, 5537, 8932, 806, 11, 611, 356, 423, 284, 4136, 20594, 986, 383, 1388, 2095, 318, 4939, 290, 7650, 78, 11, 475, 423, 366, 27659, 40024, 590, 1911, 4380, 588, 284, 8996, 11, 284, 5052, 11, 284, 13446, 13, 1374, 546, 655, 13226, 30, 40473, 1517, 1165, 11, 661, 3597, 6401, 959, 3073, 1605, 475, 11, 319, 262, 584, 1021, 11, 11810, 484, 4702, 1605, 2168, 357, 10185, 737, 6674, 340, 338, 262, 3303, 11, 393, 262, 4437, 11, 475, 314, 892, 428, 2168, 318, 517, 3594, 621, 1605, 13, 2750, 262, 835, 11, 262, 10544, 389, 1107, 922, 290, 8258, 13, 383, 7205, 318, 407, 31194, 379, 477, 986, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50

In [23]:
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: 'NEGATIVE', 1: 'POSITIVE'},
    label2id={'NEGATIVE': 0, 'POSITIVE': 1},
)

model.config.pad_token_id = tokenizer.eos_token_id
# Freeze all params
for param in model.base_model.parameters():
    param.requires_grad = False

model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [9]:
# Train model
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./data/sentiment_analysis',
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.484657,0.780000
2,No log,0.451271,0.786000
3,No log,0.453535,0.778000
4,0.524500,0.463260,0.766000
5,0.524500,0.452575,0.778000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-375 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-625 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=625, training_loss=0.4920961120605469, metrics={'train_runtime': 494.2062, 'train_samples_per_second': 5.059, 'train_steps_per_second': 1.265, 'total_flos': 1306483752960000.0, 'train_loss': 0.4920961120605469, 'epoch': 5.0})

In [11]:
# Evaluate performance
trainer.evaluate()

{'eval_loss': 0.4512712359428406,
 'eval_accuracy': 0.786,
 'eval_runtime': 45.7698,
 'eval_samples_per_second': 10.924,
 'eval_steps_per_second': 2.731,
 'epoch': 5.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [24]:
# Create PEFT config

from peft import LoraConfig

config = LoraConfig(
)

In [25]:
# Convert transformers model into peft model

from transformers import AutoModelForCausalLM
from peft import get_peft_model

# model = AutoModelForCausalLM.from_pretrained(
#     'gpt2',
#     num_labels=2,
#     id2label={0: 'NEGATIVE', 1: 'POSITIVE'},
#     label2id={'NEGATIVE': 0, 'POSITIVE': 1},
# )
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()
lora_model

trainable params: 294,912 || all params: 124,736,256 || trainable%: 0.236428452686603


PeftModel(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
        

In [28]:
# Train peft model

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./data/sentiment_analysis',
        learning_rate=2e-3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.316058,0.888000
2,No log,0.419187,0.880000
3,No log,0.424800,0.894000
4,0.256000,0.441950,0.890000
5,0.256000,0.452927,0.898000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-125 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-250 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-375 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/sentiment_analysis/checkpoint-625 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=625, training_loss=0.21606450119018555, metrics={'train_runtime': 801.2783, 'train_samples_per_second': 3.12, 'train_steps_per_second': 0.78, 'total_flos': 1311013601280000.0, 'train_loss': 0.21606450119018555, 'epoch': 5.0})

In [29]:
# Save model params. This only saves the adaptor weights.

lora_model.save_pretrained('./')
# lora_model.save_parameters('gpt-lora')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [31]:
# Load saved peft model

from peft import AutoPeftModelForCausalLM

lora_model = AutoPeftModelForCausalLM.from_pretrained('./')


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
lora_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear(
                in_features=768, out_features=2304, bias=True
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
             

In [33]:
# Evaluate the performance

print(trainer.evaluate())

{'eval_loss': 0.31605803966522217, 'eval_accuracy': 0.888, 'eval_runtime': 48.3098, 'eval_samples_per_second': 10.35, 'eval_steps_per_second': 2.587, 'epoch': 5.0}


## Performance
Accuracy prior to fine tuning: 0.786

Accuracy after fine tuning: 0.888